In [7]:
import numpy as np
import pandas as pd 
import requests 
from bs4 import BeautifulSoup
import mysql.connector
from sqlalchemy import create_engine

In [8]:
db_name = "World_Economies.db"
table_name = "Country_by_GDP"
csv_path = "Country_by_GDP.csv"